In [1]:
import pandas as pd
import numpy as np

In [3]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
init_notebook_mode(connected=True)

### Data load

In [2]:
airport = pd.read_csv("Airports2.csv")

In [8]:
airport.columns

Index(['Origin_airport', 'Destination_airport', 'Origin_city',
       'Destination_city', 'Passengers', 'Seats', 'Flights', 'Distance',
       'Fly_date', 'Origin_population', 'Destination_population',
       'Org_airport_lat', 'Org_airport_long', 'Dest_airport_lat',
       'Dest_airport_long'],
      dtype='object')

In [16]:
data.head()

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long,Origin_state,Destination_state
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NaN,NaN,KS,IA
636,SEA,RDM,"Seattle, WA","Bend, OR",2023,4144,112,228,2000-01-01,6104758,116618,47.449001,-122.308998,44.254101,-121.150001,WA,OR
637,SEA,RDM,"Seattle, WA","Bend, OR",2995,4699,127,228,2000-12-01,6104758,116618,47.449001,-122.308998,44.254101,-121.150001,WA,OR
638,SEA,RDM,"Seattle, WA","Bend, OR",3134,5328,144,228,2000-09-01,6104758,116618,47.449001,-122.308998,44.254101,-121.150001,WA,OR
639,SEA,RDM,"Seattle, WA","Bend, OR",3420,5328,144,228,2000-06-01,6104758,116618,47.449001,-122.308998,44.254101,-121.150001,WA,OR


### Feature engineering
We are only interested in data from 2000 till 2009.

In [39]:
data = airport[airport["Fly_date"] >= '2000-01-01']

In [11]:
len(data)

2122293

In [40]:
data['Origin_state'] = data['Origin_city'].apply(lambda x: x.split(',')[1].strip(' '))
data['Origin_city'] = data['Origin_city'].apply(lambda x: x.split(',')[0].strip(' '))
data['Destination_state'] = data['Destination_city'].apply(lambda x: x.split(',')[1].strip(' '))
data['Destination_city'] = data['Destination_city'].apply(lambda x: x.split(',')[0].strip(' '))
data['Year'] = data['Fly_date'].apply(lambda x: x.split('-')[0].strip(' '))
data['Month'] = data['Fly_date'].apply(lambda x: x.split('-')[1].strip(' '))

/Users/leesa/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/leesa/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/leesa/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

Let's have a look at the average daily passengers from each state in 2009.

In [78]:
df = data[data["Fly_date"] >= '2009-01-01']
df = df[['Origin_state', 'Passengers', 'Fly_date']]
df = df.groupby(['Origin_state', 'Fly_date'])['Passengers'].sum().reset_index()
df.head()

,Origin_state,Fly_date,Passengers
0,AK,2009-01-01,154555
1,AK,2009-02-01,141812
2,AK,2009-03-01,170902
3,AK,2009-04-01,154018
4,AK,2009-05-01,196890


In [84]:
df = df.groupby(['Origin_state'])['Passengers'].mean().reset_index()
df.head()

,Origin_state,Passengers
0,AK,2.084442e+05
1,AL,2.115337e+05
2,AR,1.436639e+05
3,AZ,1.532344e+06
4,CA,4.685130e+06


In [85]:
for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['Origin_state'] + '<br>' +\
    'Passengers '+ df['Passengers']

fig_data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['Origin_state'],
        z = df['Passengers'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Avg passengers")
        ) ]

layout = dict(
        title = '2009 Average passengers by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict(data=fig_data, layout=layout)

iplot(fig)

It seems there are a lot of passengers from California and Texas travelling to other states.  
Now lets have a look at the change in average air passengers across the years for these two states.

In [90]:
df = data[['Origin_state', 'Passengers', 'Fly_date', 'Year']]
df = df.groupby(['Origin_state', 'Fly_date', 'Year'])['Passengers'].sum().reset_index()
df = df.groupby(['Origin_state', 'Year'])['Passengers'].mean().reset_index()
df.head()

,Origin_state,Year,Passengers
0,AK,2000,197716.333333
1,AK,2001,197246.083333
2,AK,2002,208416.416667
3,AK,2003,205828.083333
4,AK,2004,221796.333333


In [97]:
data1 = go.Scatter(
        x=df[df['Origin_state'] == 'CA']['Year'], 
        y=df[df['Origin_state'] == 'CA']['Passengers']
    )

data2 = go.Scatter(
        x=df[df['Origin_state'] == 'TX']['Year'], 
        y=df[df['Origin_state'] == 'TX']['Passengers']
    )

fig_data = [data1, data2]
fig = go.Figure(data = fig_data)
iplot(fig)

For both the states the change is almost similar. The number decreases till 2002 and then increases until 2007 after which it again starts decreasing. This similarity increased our curiosity to check how the overall national average looks like.

In [43]:
df3 = data[['Passengers', 'Fly_date', 'Year']]
df3 = df3.groupby(['Fly_date', 'Year'])['Passengers'].sum().reset_index()
df3 = df3.groupby(['Year'])['Passengers'].mean().reset_index()
df3.head()

,Year,Passengers
0,2000,4.449642e+07
1,2001,4.141747e+07
2,2002,4.050238e+07
3,2003,4.241575e+07
4,2004,4.553912e+07


In [98]:
fig_data = [go.Scatter(
        x=df3['Year'], 
        y=df3['Passengers']
    )]

fig = go.Figure(data = fig_data)
iplot(fig)

To our surprise, the national average follows almost similar trend. This makes us think that there might be some driving factor for this change.

In [42]:
data[(data['Fly_date'] == '2009-01-01') & (data['Origin_airport'] == 'SFO')]

,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long,Origin_state,Destination_state,Year,Month
1648,SFO,RDM,San Francisco,Bend,1390,2610,87,462,2009-01-01,8635706,158629,37.618999,-122.375,44.254101,-121.150001,CA,OR,2009,01
29545,SFO,RNO,San Francisco,Reno,2237,2900,58,192,2009-01-01,8635706,419261,37.618999,-122.375,39.499100,-119.767998,CA,NV,2009,01
29550,SFO,RNO,San Francisco,Reno,4393,6924,46,192,2009-01-01,8635706,419261,37.618999,-122.375,39.499100,-119.767998,CA,NV,2009,01
29553,SFO,RNO,San Francisco,Reno,579,960,8,192,2009-01-01,8635706,419261,37.618999,-122.375,39.499100,-119.767998,CA,NV,2009,01
29556,SFO,RNO,San Francisco,Reno,749,990,15,192,2009-01-01,8635706,419261,37.618999,-122.375,39.499100,-119.767998,CA,NV,2009,01
38250,SFO,CIC,San Francisco,Chico,1819,3420,114,153,2009-01-01,8635706,220577,37.618999,-122.375,39.795399,-121.858002,CA,CA,2009,01
95091,SFO,MIA,San Francisco,Miami,11535,12972,69,2585,2009-01-01,8635706,11094102,37.618999,-122.375,25.793200,-80.290604,CA,FL,2009,01
95092,SFO,MIA,San Francisco,Miami,4368,4950,22,2585,2009-01-01,8635706,11094102,37.618999,-122.375,25.793200,-80.290604,CA,FL,2009,01
208187,SFO,AUS,San Francisco,Austin,2378,3120,26,1504,2009-01-01,8635706,1705075,37.618999,-122.375,30.194500,-97.669899,CA,TX,2009,01
208193,SFO,AUS,San Francisco,Austin,577,690,5,1504,2009-01-01,8635706,1705075,37.618999,-122.375,30.194500,-97.669899,CA,TX,2009,01


In [25]:
df = data[data['Fly_date'] == '2009-01-01'][['Origin_state', 'Origin_airport',\
                                             'Flights', 'Origin_city', 'Org_airport_lat', 'Org_airport_long']]
#groupby(['Origin_state', 'Origin_airport'])['Flights'].sum().reset_index()
df.head()

,Origin_state,Origin_airport,Flights,Origin_city,Org_airport_lat,Org_airport_long
1511,CA,LAX,29,Los Angeles,33.942501,-118.407997
1539,OR,MFR,1,Medford,42.374199,-122.873001
1540,OR,MFR,1,Medford,42.374199,-122.873001
1543,OR,MFR,1,Medford,42.374199,-122.873001
1548,WA,SEA,2,Seattle,47.449001,-122.308998


In [26]:
df = df.groupby(['Origin_state', 'Origin_airport',
                 'Origin_city', 'Org_airport_lat', 'Org_airport_long'])['Flights'].sum().reset_index()

In [32]:
df['text'] = df['Origin_airport'] + '<br>' + df['Origin_city'] + ', ' + df['Origin_state'] + '<br>' + 'Takeoffs: ' + df['Flights'].astype(str)

scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

fig_data = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df['Org_airport_long'],
        lat = df['Org_airport_lat'],
        text = df['text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = df['Flights'],
            cmax = df['Flights'].max(),
            colorbar=dict(
                title="Incoming flights in a day in 2009"
            )
        ))]

layout = dict(
        title = 'Most trafficked US airports<br>(Hover for airport names)',
        colorbar = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=fig_data, layout=layout )
iplot(fig, validate=False)

The plot above shows the number of flights took of from each airport in the USA.